In [39]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as ur
from bbmag import bb_abmag, sigerr
from tdsat_sensitivity import find_limit, compute_snr
from tdsat_telescope import load_telescope_parameters
from astropy.visualization import quantity_support
from astropy.modeling import models, fitting
from astropy.modeling.blackbody import FLAM, FNU
from scipy.optimize import curve_fit

%matplotlib inline

In [2]:
plt.rcParams['figure.figsize'] = [15,10]

In [3]:
# Setup:
td1_band = [180,220]*ur.nm

td2_bandwidth = [40,60,80,100,120,140,160,180,200]
td2 = []
for w in td2_bandwidth:
    td2.append([260,260+w])
    
td2_band = td2*ur.nm
    
bbtemp = [15000,20000,30000]*ur.K

dist = [100,200]*ur.Mpc

# For the shock model after 1 hr:
swiftmag_sh = -16.5*ur.ABmag

# For the blue kilonova model after 1 hr:
swiftmag_bk = -13.6*ur.ABmag

# Satellite details:
diameter, qe, psf_fwhm, efficiency = load_telescope_parameters(2, diag=True)
outofband_qe = 0.001
exposure = 300*ur.s

Telescope Configuration 2
Entrance Pupil diameter 30.0 cm
Optical Effifiency 0.65
PSF FWHM 9.0 arcsec
Effective Aperture 24.18677324489565 cm
Effective Area 459.45792558750725 cm2


In [18]:
bbmag = bb_abmag(bbtemp=bbtemp[0], bandone=td1_band, 
                                  bandtwo=td2_band[0], swiftmag=swiftmag_sh, 
                                  dist=dist[0], val=True)
bbmag = bbmag*ur.ABmag

In [19]:
snr = [0,0]
snr[0] = compute_snr(td1_band,bbmag[0],
                                  low_zodi=True,diameter=diameter,
                                  exposure = exposure,
                                  efficiency = efficiency, qe = qe,
                                  outofband_qe = outofband_qe, psf_size = psf_fwhm)

snr[1] = compute_snr(td2_band[0],bbmag[1],
                                  low_zodi=True,diameter=diameter,
                                  exposure = exposure,
                                  efficiency = efficiency, qe = qe,
                                  outofband_qe = outofband_qe, psf_size = psf_fwhm)

sig = 2.5*np.log10(1.+1./np.array(snr))

In [62]:
bb_init = models.BlackBody1D(temperature=10000*ur.K)
fit_bb = fitting.LevMarLSQFitter()
bb_fden = bbmag.to(FLAM,equivalencies=ur.spectral_density([200,280]*ur.nm))
bb_fnu = bbmag.to(FNU,equivalencies=ur.spectral_density([200,280]*ur.nm))
bb_err = bb_fden/snr

In [48]:
bb = fit_bb(bb_init,[200,280]*ur.nm,bb_fnu)

In [64]:
print(bb_fden,bb_err)

[1.06367987e-15 8.04506008e-16] erg / (Angstrom cm2 s) [1.59286268e-17 1.19894451e-17] erg / (Angstrom cm2 s)


In [49]:
print(bb)

Model: BlackBody1D
Inputs: ('x',)
Outputs: ('y',)
Model set size: 1
Parameters:
       temperature        bolometric_flux    
            K              erg / (cm2 s)     
    ------------------ ----------------------
    14907.214006153004 3.1505847372874166e-12


In [56]:
bbfunc(200,10000,1.)

0.00015501623025437233

In [52]:
bb_init(200*ur.nm).to(FLAM,equivalencies=ur.spectral_density(200*ur.nm))

<Quantity 0.00015502 erg / (Angstrom cm2 s)>

In [55]:
def bbfunc(x,*par):
    temp,norm = par
    mod = models.BlackBody1D(temperature=temp*ur.K,bolometric_flux = norm*ur.erg/(ur.cm**2 * ur.s))
    return mod(x*ur.nm).to(FLAM, ur.spectral_density(x*ur.nm)).value

In [65]:
coeff, var_matrix = curve_fit(bbfunc,[200,280],bb_fden.value,p0=[10000,3.E-12],sigma=bb_err.value,absolute_sigma=True)

In [66]:
coeff

array([1.49072140e+04, 3.15058474e-12])

In [67]:
var_matrix

array([[ 5.82927999e+04, -2.58006048e-12],
       [-2.58006048e-12,  1.22179824e-27]])

In [68]:
perr = np.sqrt(np.diag(var_matrix))

In [69]:
perr

array([2.41439019e+02, 3.49542307e-14])

In [73]:
test = [21,22]*ur.ABmag

In [74]:
test_fden = test.to(FLAM,equivalencies=ur.spectral_density([200,280]*ur.nm))